# Data wrangling WeRateDogs DAND project




## Table of Contents
- 1 [Introduction](#intro)
- 2 [Gathering data](#gathering)
- 3 [Assessing data](#assessing)
- 4 [Cleaning data](#cleaning)
 - 4.1 [Issue #1](#issue1)
 - 4.2 [Issue #2](#issue2)
 - 4.3 [Issue #3](#issue3)
 - 4.4 [Issue #4](#issue4)
 - 4.5 [Issue #5](#issue5)
 - 4.6 [Issue #6](#issue6)
 - 4.7 [Issue #7](#issue7)
 - 4.8 [Issue #8](#issue8)
 - 4.9 [Issue #9](#issue9)
 - 4.10 [Issue #10](#issue10)
 - 4.11 [Issue #11](#issue11)
 - 4.12 [Additional](#additional)

<a id='intro'></a>
## Introduction

The goal of this project is to wrangle the WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The challenge is that the Twitter archive is great, but it only contains very basic tweet information that comes in JSON format. For a successful project, I needed to gather, asses and clean the Twitter data for a worthy analysis and visualization.

The dataset that we will be wrangling, analyzing and visualizing is the tweet archive of Twitter user [@dog_rates](https://twitter.com/dog_rates)


<a id='gathering'></a>
## Gathering data

In this project we will gather Data from three sources as described below:

- The WeRateDogs Twitter archive. We will download this file manually from:  [twitter_archive_enhanced.csv](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv)
- The tweet image predictions, this file (image_predictions.tsv) hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
- Tweet's retweet count and favorite (tweet.json.txt) I downloaded from udacity because I couldn't gain twitter developer account- as described in project details (Twitter API) section if I cannot get this file from twitter I can proceed with the "Gathering Data" ,Then read this tweet_json.txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count."


#### Importing the dataset

In [1]:
# Import the libraries that we will need in this project
import pandas as pd
import datetime as dt
import numpy as np
import requests
import tweepy
import json
import re
import time
from nltk import pos_tag

Now we will import the data we have in format of csv:

In [2]:
# Read the twitter-archive-enhanced.csv file and store it as dataframe in archive
archive = pd.read_csv('twitter-archive-enhanced.csv')
# Quick check to the file content and structure
archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [3]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

> **The file twitter-archive-enhanced.csv successfully stored in archive dataframe and it has 17 columns and 2356 entries**

In [4]:
 # Read the images file
images = pd.read_csv('image-predictions.tsv', sep = '\t', encoding = 'utf-8')
# Quick check to the file content and structure
images

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [5]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


> **The file image-predictions.tsv successfully downloaded and stored in images dataframe, it has 12 columns and 2075 entries**

In [6]:
# import tweepy
# from tweepy import OAuthHandler
# import json
# from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
     # consumer_key = 'HIDDEN'
     # consumer_secret = 'HIDDEN'
     # access_token = 'HIDDEN'
     # access_secret = 'HIDDEN'

     # auth = OAuthHandler(consumer_key, consumer_secret)
     # auth.set_access_token(access_token, access_secret)

     # api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
    # tweet_ids = archive.tweet_id.values
    # len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
    # count = 0
    # fails_dict = {}
    # start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
    # with open('tweet_json.txt', 'w') as outfile:
# This loop will likely take 20-30 minutes to run because of Twitter's rate limit
       # for tweet_id in tweet_ids:
        # count += 1
        # print(str(count) + ": " + str(tweet_id))
         # try:
            #tweet = api.get_status(tweet_id, tweet_mode='extended')
            # print("Success")
            # json.dump(tweet._json, outfile)
            # outfile.write('\n')
         # except tweepy.TweepError as e:
            # print("Fail")
            # fails_dict[tweet_id] = e
            # pass
 # end = timer()
 # print(end - start)
 # print(fails_dict)

In [7]:
import pandas as pd
import json

# Read tweet_json.txt file line by line into a pandas dataframe
df = pd.DataFrame()

with open('tweet-json.txt', 'r') as f:
# Read in JSON line and convert to dict
    lines =f.readlines()
# Read line by line into a dataframe
    jsonStr = "[" + "," .join(lines) + "]"

    all_dicts = json.loads(jsonStr)

    for i in all_dicts:

        df = df.append({key:i[key] for key in ['id_str','retweet_count','favorite_count']},ignore_index=True)

In [8]:
# Convert the dataFrame to csv file
df.to_csv('tweet-json.csv',index=False)

In [9]:
# Read the tweet-json.csv file
tweet = pd.read_csv('tweet-json.csv')
# Quick check to the file content and structure
tweet

,favorite_count,id_str,retweet_count
0,39467.0,892420643555336193,8853.0
1,33819.0,892177421306343426,6514.0
2,25461.0,891815181378084864,4328.0
3,42908.0,891689557279858688,8964.0
4,41048.0,891327558926688256,9774.0
5,20562.0,891087950875897856,3261.0
6,12041.0,890971913173991426,2158.0
7,56848.0,890729181411237888,16716.0
8,28226.0,890609185150312448,4429.0
9,32467.0,890240255349198849,7711.0


In [10]:
tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null float64
id_str            2354 non-null int64
retweet_count     2354 non-null float64
dtypes: float64(2), int64(1)
memory usage: 55.2 KB


> **The file tweet_json.txt successfully saved in our working directory in csv format and its contain 3 columns and 2354 rows.**

#### Gather: Summary

Gathering data is the **first step** in the data wrangling process and we gathered the data from various resources:

- Getting data from an existing file (twitter-archive-enhanced.csv) Reading from csv file using pandas.
- Downloading a file from the internet (image-predictions.tsv) Downloading file using requests.
- Read tweet_json.txt file line by line into a pandas dataframe then convert to csv file. 

<a id='assessing'></a>
## Assessing data

After gathering the data above, assess this data will be the next step to be sure that the quality of this data is suitable to move to last step which is the cleaning step.

In [11]:
# Print all archive dataset to assess it visually
archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [12]:
# Print some random expamles from columns values, random number just to check if we can find something sucpisious
print(archive['text'][777])
print(archive['name'][7])

I can't tap the screen to make the hearts appear fast enough. 10/10 for the source of all future unproductiveness https://t.co/wOhuABgj6I
None


In [13]:
# Assessing the data programmaticaly
archive.info()
archive.describe()
archive['rating_numerator'].value_counts()
archive['rating_denominator'].value_counts()
archive['name'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

None        745
a            55
Charlie      12
Cooper       11
Oliver       11
Lucy         11
Lola         10
Penny        10
Tucker       10
Bo            9
Winston       9
Sadie         8
the           8
an            7
Toby          7
Bailey        7
Daisy         7
Buddy         7
Milo          6
Dave          6
Koda          6
Stanley       6
Rusty         6
Oscar         6
Scout         6
Leo           6
Jax           6
Jack          6
Bella         6
George        5
           ... 
Harry         1
Emma          1
Millie        1
Hermione      1
Chuck         1
Ember         1
Lassie        1
Lizzie        1
Duchess       1
Cora          1
Buddah        1
Ridley        1
Nimbus        1
Noah          1
Perry         1
BeBe          1
Roscoe        1
Murphy        1
Tug           1
Fwed          1
Julius        1
Furzey        1
Divine        1
Joshwa        1
Bauer         1
Reagan        1
Wiggles       1
Severus       1
Kevon         1
Arya          1
Name: name, Length: 957,

In [14]:
# Print all images dataset to assess it visually
images

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [15]:
images.info()
images['jpg_url'].value_counts()
images[images['jpg_url'] == 'https://pbs.twimg.com/ext_tw_video_thumb/675354114423808004/pu/img/qL1R_nGLqa6lmkOx.jpg']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
480,675354435921575936,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True


In [16]:
# Print all tweet dataset to assess it visually
tweet

,favorite_count,id_str,retweet_count
0,39467.0,892420643555336193,8853.0
1,33819.0,892177421306343426,6514.0
2,25461.0,891815181378084864,4328.0
3,42908.0,891689557279858688,8964.0
4,41048.0,891327558926688256,9774.0
5,20562.0,891087950875897856,3261.0
6,12041.0,890971913173991426,2158.0
7,56848.0,890729181411237888,16716.0
8,28226.0,890609185150312448,4429.0
9,32467.0,890240255349198849,7711.0


In [17]:
# Get tweet.csv information
tweet.info

<bound method DataFrame.info of       favorite_count              id_str  retweet_count
0            39467.0  892420643555336193         8853.0
1            33819.0  892177421306343426         6514.0
2            25461.0  891815181378084864         4328.0
3            42908.0  891689557279858688         8964.0
4            41048.0  891327558926688256         9774.0
5            20562.0  891087950875897856         3261.0
6            12041.0  890971913173991426         2158.0
7            56848.0  890729181411237888        16716.0
8            28226.0  890609185150312448         4429.0
9            32467.0  890240255349198849         7711.0
10           31166.0  890006608113172480         7624.0
11           28268.0  889880896479866881         5156.0
12           38818.0  889665388333682689         8538.0
13           27672.0  889638837579907072         4735.0
14           15359.0  889531135344209921         2321.0
15           25652.0  889278841981685760         5637.0
16           296

In [18]:
tweet.id_str.sort_values()

2353    666020888022790149
2352    666029285002620928
2351    666033412701032449
2350    666044226329800704
2349    666049248165822465
2348    666050758794694657
2347    666051853826850816
2346    666055525042405380
2345    666057090499244032
2344    666058600524156928
2343    666063827256086533
2342    666071193221509120
2341    666073100786774016
2340    666082916733198337
2339    666094000022159362
2338    666099513787052032
2337    666102155909144576
2336    666104133288665088
2335    666268910803644416
2334    666273097616637952
2333    666287406224695296
2332    666293911632134144
2331    666337882303524864
2330    666345417576210432
2329    666353288456101888
2328    666362758909284353
2327    666373753744588802
2326    666396247373291520
2325    666407126856765440
2324    666411507551481857
               ...        
29      886267009285017600
28      886366144734445568
27      886680336477933568
26      886736880519319552
25      886983233522544640
24      887101392804085760
2

> **Some description of the variables in tweet dataset:**
- id_str: its representation of the unique identifier for this Tweet.
- retweet_count: Number of times this Tweet has been retweeted.
- favorite_count: Indicates approximately how many times this Tweet has been liked by Twitter users.

### Quality

#### content issues:

**archive dataset:**

- Timestamp,in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id should be we will change the data type of all columns.
- In several columns null objects are non-null (None to NaN)
- Some entries have invalid values i.e 'None', 'a', 'an'
- The ratings are not extracted correctly some have error.


**images dataset:**

- Some tweet_ids have the same jpg_url
- Some tweets are having 2 different tweet_id one redirect to the other


**tweeet dataset:**

- The tweet_id column should be named same in all the DataFrames and its datatype should be same in all the tables (id_str column name to tweet_id so I can merge the file with other files [to be a key column in all dataset].

**In all dataset:**

- The name column has many invalid values like , a, an, the.

### Tidiness

#### structural issues

- No need to all the informations in images dataset, (tweet_id and jpg_url what matters)
- Merge 4 columns into 1 column (Merge the 'doggo', 'floofer', 'pupper' and 'puppo' columns into one column named 'dog_stage') in archives dataset.
- All tables should be part of one dataset

### Additional 

- We may want to add a gender column from the text columns in archives dataset

<a id='cleaning'></a>
## Cleaning data

Cleaning our data is the third step in data wrangling. It is where we will fix the quality and tidiness issues that we identified in the assess step.

### Renamed the Id_str column to be same in all the DataFrames (tweet_id)

<a id='issue1'></a>
**4.1.1 DEFINE ISSUE #1**

**First**
I will change the id_str column name in tweet dataset to be match with tweet_id column in other dataset:

**4.1.2 CLEAN ISSUE #1**

In [19]:
# making data frame from csv file 
tweet = pd.read_csv("tweet-json.csv") 
  
# changing index cols with rename() 
tweet.rename(columns = {"id_str": "tweet_id"}, inplace = True) 

**4.1.3 TEST CLEANED ISSUE #1**

In [20]:
# check the column new name 
tweet

,favorite_count,tweet_id,retweet_count
0,39467.0,892420643555336193,8853.0
1,33819.0,892177421306343426,6514.0
2,25461.0,891815181378084864,4328.0
3,42908.0,891689557279858688,8964.0
4,41048.0,891327558926688256,9774.0
5,20562.0,891087950875897856,3261.0
6,12041.0,890971913173991426,2158.0
7,56848.0,890729181411237888,16716.0
8,28226.0,890609185150312448,4429.0
9,32467.0,890240255349198849,7711.0


### Collect all dataset in one dataset named df_master

<a id='issue1'></a>
**4.2.1 DEFINE ISSUE #2**

Here we will merge all dataset in one dataset.

**4.2.2 CLEAN ISSUE #2**

In [21]:
# Since we want to create one high quality and tidy master pandas DataFrame we will start by merging our dataframe in one
df_master = pd.merge(archive, images, how = 'left', on = ['tweet_id'] )
df_master = pd.merge(df_master, tweet, how = 'left', on = ['tweet_id'])
df_master.to_csv('df_master.csv', encoding = 'utf-8')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
jpg_url                       20

**4.2.3 TEST CLEANED ISSUE #2**

In [22]:
df_master

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,favorite_count,retweet_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,0.097049,False,bagel,0.085851,False,banana,0.076110,False,39467.0,8853.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,33819.0,6514.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,...,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,25461.0,4328.0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,...,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,42908.0,8964.0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,...,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,41048.0,9774.0
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,...,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,20562.0,3261.0
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",...,0.341703,True,Border_collie,0.199287,True,ice_lolly,0.193548,False,12041.0,2158.0
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,...,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True,56848.0,16716.0
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,...,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True,28226.0,4429.0
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,...,0.511319,True,Cardigan,0.451038,True,Chihuahua,0.029248,True,32467.0,7711.0


> **We now have 30 columns and 2356 entries.** 

### Quick Clean to rows and columns that we will not need

<a id='issue1'></a>
**4.3.1 DEFINE ISSUE #3**

We have many columns that we will not use or we didn't want it, so we will rwmove it from our dataset.

**4.3.2 CLEAN ISSUE #3**

In [23]:
# Delete the retweeted
df_master = df_master[pd.isnull(df_master.retweeted_status_id)]
# Delete duplicated tweet_id
df_master = df_master.drop_duplicates()
# Delete tweets with no pictures
df_master = df_master.dropna(subset = ['jpg_url'])

# test
len(df_master)

1994

In [24]:
# Delete columns related to retweet we don't need anymore
df_master = df_master.drop('retweeted_status_id', 1)
df_master = df_master.drop('retweeted_status_user_id', 1)
df_master = df_master.drop('retweeted_status_timestamp', 1)

**4.3.3 TEST CLEANED ISSUE #3**

In [25]:
# test
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'favorite_count',
 'retweet_count']

### Merge the 'doggo', 'floofer', 'pupper' and 'puppo' columns into one column 'dog_stage'

<a id='issue4'></a>
**4.4.1 DEFINE ISSUE #4**

Merge 4 columns into 1 column (Merge the 'doggo', 'floofer', 'pupper' and 'puppo' columns into one column named 'dog_stage').

**4.4.2 CLEAN ISSUE #4**

In [26]:
# Check the values in specific columns by excuting those columns
print(df_master.doggo.value_counts())
print(df_master.floofer.value_counts())
print(df_master.pupper.value_counts())
print(df_master.puppo.value_counts())

None     1920
doggo      74
Name: doggo, dtype: int64
None       1986
floofer       8
Name: floofer, dtype: int64
None      1782
pupper     212
Name: pupper, dtype: int64
None     1971
puppo      23
Name: puppo, dtype: int64


In [27]:
# Select the columns to merge and to remain

columns_to_melt = ['doggo', 'floofer', 'pupper', 'puppo']

 

def a(row):

    x=[]

    for i in row:

         if i != 'None':

             x.append(i)

    return ','.join(x)

df_master['dog_stage'] = df_master[columns_to_melt].apply(a, axis=1) 

for i in columns_to_melt:

    df_master =df_master.drop(i, axis=1)

In [28]:
# Filter for unique values then remove duplicate values based on 'dog_stage' values
# Test
print(df_master.dog_stage.value_counts())

df_master = df_master.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

                 1688
pupper            203
doggo              63
puppo              22
doggo,pupper        9
floofer             7
doggo,puppo         1
doggo,floofer       1
Name: dog_stage, dtype: int64


**4.4.3 TEST CLEANED ISSUE #4**

In [29]:
# Test
print(df_master.dog_stage.value_counts())
print(len(df_master))

                 1688
pupper            203
doggo              63
puppo              22
doggo,pupper        9
floofer             7
doggo,puppo         1
doggo,floofer       1
Name: dog_stage, dtype: int64
1994


### Get rid of image prediction columns

<a id='issue5'></a>
**4.5.1 DEFINE ISSUE #5**

Replace the empty values with NaN, and delete some values that are unuseful.

**4.5.2 CLEAN ISSUE #5**

In [30]:
# We will store the fisrt true algorithm with it's level of confidence
prediction_algorithm = []
confidence_level = []

# Get_prediction_confidence function:
# search the first true algorithm and append it to a list with it's level of confidence
# if flase prediction_algorthm will have a value of NaN
def get_prediction_confidence(dataframe):
    if dataframe['p1_dog'] == True:
        prediction_algorithm.append(dataframe['p1'])
        confidence_level.append(dataframe['p1_conf'])
    elif dataframe['p2_dog'] == True:
        prediction_algorithm.append(dataframe['p2'])
        confidence_level.append(dataframe['p2_conf'])
    elif dataframe['p3_dog'] == True:
        prediction_algorithm.append(dataframe['p3'])
        confidence_level.append(dataframe['p3_conf'])
    else:
        prediction_algorithm.append('NaN')
        confidence_level.append(0)

df_master.apply(get_prediction_confidence, axis=1)
df_master['prediction_algorithm'] = prediction_algorithm
df_master['confidence_level'] = confidence_level

# Test
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'favorite_count',
 'retweet_count',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level']

In [31]:
# Delete the columns of image prediction information
df_master = df_master.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], 1)

**4.5.3 CLEAN ISSUE #5**

In [32]:
# Test
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'jpg_url',
 'favorite_count',
 'retweet_count',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level']

### Check for duplicated values then delete useless informations

<a id='issue6'></a>
**4.6.1 DEFINE ISSUE #6**

Here I will check if there is any duplication in the dataset and delete the information that we will not use it.

**4.6.2 CLEAN ISSUE #6**

In [33]:
# Print the count of the unique elements in all columns
df_master.apply(lambda x: len(x.unique()))

tweet_id                 1994
in_reply_to_status_id      23
in_reply_to_user_id         2
timestamp                1994
source                      3
text                     1994
expanded_urls            1994
rating_numerator           34
rating_denominator         15
name                      936
jpg_url                  1994
favorite_count           1850
retweet_count            1595
dog_stage                   8
prediction_algorithm      114
confidence_level         1684
dtype: int64

In [34]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 961
Data columns (total 16 columns):
tweet_id                 1994 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                1994 non-null object
source                   1994 non-null object
text                     1994 non-null object
expanded_urls            1994 non-null object
rating_numerator         1994 non-null int64
rating_denominator       1994 non-null int64
name                     1994 non-null object
jpg_url                  1994 non-null object
favorite_count           1994 non-null float64
retweet_count            1994 non-null float64
dog_stage                1994 non-null object
prediction_algorithm     1994 non-null object
confidence_level         1994 non-null float64
dtypes: float64(5), int64(3), object(8)
memory usage: 264.8+ KB


In [35]:
df_master['in_reply_to_user_id'].value_counts()

4.196984e+09    23
Name: in_reply_to_user_id, dtype: int64

In [36]:
df_master['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     1955
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       28
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

**From the above we find:**
- One value in in_reply_to_user_id so we will delete the columns of reply all of them replying to one user id '4196983835'
- Source has 3 types, we will clean that column and made them clear

In [37]:
# Delete 'in_reply_to_status_id', 'in_reply_to_user_id' columns
df_master = df_master.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], 1)

In [38]:
# Clean the content of source column
df_master['source'] = df_master['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

**4.6.3 TEST CLEANED ISSUE #6**

In [39]:
# Test
df_master

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,favorite_count,retweet_count,dog_stage,prediction_algorithm,confidence_level
0,892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,39467.0,8853.0,,NaN,0.000000
1643,683857920510050305,2016-01-04 03:50:08 +0000,Twitter for iPhone,Meet Sadie. She fell asleep on the beach and h...,https://twitter.com/dog_rates/status/683857920...,10,10,Sadie,https://pbs.twimg.com/media/CX2NJmRWYAAxz_5.jpg,4163.0,1262.0,,bluetick,0.174738
1642,684097758874210310,2016-01-04 19:43:10 +0000,Twitter for iPhone,Say hello to Lupe. This is how she sleeps. 10/...,https://twitter.com/dog_rates/status/684097758...,10,10,Lupe,https://pbs.twimg.com/media/CX5nR5oWsAAiclh.jpg,4515.0,1621.0,,Labrador_retriever,0.627856
1641,684122891630342144,2016-01-04 21:23:02 +0000,Twitter for iPhone,Heartwarming scene of two pups that want nothi...,https://twitter.com/dog_rates/status/684122891...,11,10,None,https://pbs.twimg.com/media/CX5-HslWQAIiXKB.jpg,2176.0,539.0,,NaN,0.000000
1639,684177701129875456,2016-01-05 01:00:50 +0000,Twitter for iPhone,This is Kulet. She's very proud of the flower ...,https://twitter.com/dog_rates/status/684177701...,10,10,Kulet,https://pbs.twimg.com/media/CX6v_JOWsAE0beZ.jpg,2215.0,764.0,,chow,0.334783
1638,684188786104872960,2016-01-05 01:44:52 +0000,Twitter for iPhone,"""Yo Boomer I'm taking a selfie, grab your stic...",https://twitter.com/dog_rates/status/684188786...,10,10,None,https://pbs.twimg.com/media/CX66EiJWkAAVjA-.jpg,3831.0,1343.0,,kelpie,0.537782
1637,684195085588783105,2016-01-05 02:09:54 +0000,Twitter for iPhone,This is Tino. He really likes corndogs. 9/10 h...,https://twitter.com/dog_rates/status/684195085...,9,10,Tino,https://pbs.twimg.com/media/CX6_y6OU0AAl3v2.jpg,2108.0,595.0,,Chihuahua,0.379365
1636,684200372118904832,2016-01-05 02:30:55 +0000,Twitter for iPhone,Gang of fearless hoofed puppers here. Straight...,https://twitter.com/dog_rates/status/684200372...,6,10,None,https://pbs.twimg.com/media/CX7EkuHWkAESLZk.jpg,2394.0,1177.0,,NaN,0.000000
1635,684222868335505415,2016-01-05 04:00:18 +0000,Twitter for iPhone,Someone help the girl is being mugged. Several...,https://twitter.com/dog_rates/status/684222868...,121,110,None,https://pbs.twimg.com/media/CX7Y_ByWwAEJdUy.jpg,4225.0,1563.0,,soft-coated_wheaten_terrier,0.791182
1634,684225744407494656,2016-01-05 04:11:44 +0000,Twitter for iPhone,"Two sneaky puppers were not initially seen, mo...",https://twitter.com/dog_rates/status/684225744...,143,130,None,https://pbs.twimg.com/media/CX7br3HWsAAQ9L1.jpg,1369.0,239.0,,golden_retriever,0.203249


###  Correct the rating and delete the error values

<a id='issue7'></a>
**4.7.1 DEFINE ISSUE #7**

If I find any error entries we will delete it and correct the wrong enteries.

**4.7.2 CLEAN ISSUE #7**

In [40]:
# Print the values and check if there exist in the text
df_master.rating_numerator.value_counts()
df_master.rating_denominator.value_counts()
print(df_master[df_master.rating_denominator == 170]['text'][1120])
print(df_master[df_master.rating_numerator == 1776]['text'][979])
print(df_master[df_master.tweet_id == 786709082849828864]['text'][695])
print(df_master['text'][2279])
print(df_master['text'][2278])
print(df_master['text'][2277])
print(df_master['text'][2272])

Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh
This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS
This is Biden. Biden just tripped... 7/10 https://t.co/3Fm9PwLju1
Ermergerd 12/10 https://t.co/PQni2sjPsm
Never seen this breed before. Very pointy pup. Hurts when you cuddle. Still cute tho. 10/10 https://t.co/97HuBrVuOx
Two dogs in this one. Both are rare Jujitsu Pythagoreans. One slightly whiter than other. Long legs. 7/10 and 8/10 https://t.co/ITxxcc4v9y


In [41]:
# Get ratings and treat them depending to their situation
ratings = df_master['text'].apply(lambda x: re.findall(r'(\d+(\.\d+)|(\d+))\/(\d+0)', x))
len(ratings)




1994

In [42]:
rating_numerator = []
rating_denominator = []
dogs_count = []

for rate in ratings:
    if len(rate) == 0:
        rating_numerator.append('NaN')
        rating_denominator.append('NaN')
        dogs_count.append(1) # It has a picture so it is a dog
            
    elif len(rate) == 1:
        rating_numerator.append((float(rate[0][0]) / (float(rate[0][-1])/10)))
        rating_denominator.append(float(rate[0][-1]))
        dogs_count.append(float(rate[0][-1]) / 10)
        
   # we take the average of the tweet with more than one rating
    elif len(rate) > 1 and rate[0][-1] == '10':
        rating_plus = 0
        rating_avg = 0
        for i in range(len(rate)):
            rating_plus = rating_plus + float(rate[i][0])
        result_avg = (rating_plus / len(rate))
        rating_numerator.append(result_avg)
        rating_denominator.append(10)
        dogs_count.append(len(rate))
        
    else: # without this block I get ValueError: Length of values does not match length of index
          # We will try to catch the errors this  why and see why this happend
        rating_numerator.append('Error')
        rating_denominator.append('Error')
        dogs_count.append('Error')

df_master['new_rating_numerator'] = rating_numerator
df_master['new_rating_denominator'] = rating_denominator
df_master['dogs_count'] = dogs_count
df_master['new_rating_numerator'].value_counts()

12.0                 453
10.0                 411
11.0                 399
13.0                 261
9.0                  152
8.0                   94
7.0                   52
14.0                  36
6.0                   32
5.0                   30
3.0                   19
4.0                   14
2.0                    9
8.5                    4
1.0                    4
7.5                    3
9.5                    3
Error                  2
10.5                   2
0.0                    2
1776.0                 1
9.75                   1
4.5                    1
5.5                    1
6.5                    1
11.26                  1
13.5                   1
9.666666666666666      1
420.0                  1
11.5                   1
NaN                    1
11.27                  1
Name: new_rating_numerator, dtype: int64

> - We still see two big numbers one of them for Atticus and the other for Good dogg we will fix them manually becaause I cannot fix it programmatically.
> - We will remove the NaN and Error values later.

In [43]:
# I write this code to determine if the rate bigger then 10 it will be change to 11,
# but the code not work very good it convert all numbers to the range of (6, 9, 10, 11, 12, 13)

# rates = [None] * len(df_master)

# for i in range(len(df_master)):

    # if i > 10:

         # rates[i] = 11

    # else:

        # rates[i] = df_master.iloc[i].rating_numerator

# df_master['rating_numerator'] = rates

In [44]:
# Delete the old columns and update the names of the new columns
df_master = df_master.drop(['rating_numerator', 'rating_denominator'], 1)

# Rename columns
df_master.rename(columns = {'new_rating_numerator': 'rating_numerator', 
                            'new_rating_denominator': 'rating_denominator'}, inplace = True)

**4.7.3 TEST CLEANED ISSUE #7**

In [45]:
# Test
list(df_master)

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'name',
 'jpg_url',
 'favorite_count',
 'retweet_count',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level',
 'rating_numerator',
 'rating_denominator',
 'dogs_count']

### Fill empty names and Correct the bad ones

<a id='issue8'></a>
**4.8.1 DEFINE ISSUE #8**

**The examples we could notice from assessing the data visually:**

- This is [name] ..
- Meet [name] ..
- Say hallo to [name] ..
- Here we have [name] ..
- .. named [name] ..

We will treat those cases to get the names from the text of the tweet

**4.8.2 CLEAN ISSUE #8**

In [46]:
# Loop on all the texts and check if the comment has one of the above conditions
# and append the result in a list
dog_names = []

for text in df_master['text']:
    # Start with 'This is ' and the fisrt letter of the name is uppercase
    if text.startswith('This is ') and re.match(r'[A-Z].*', text.split()[2]):
        dog_names.append(text.split()[2].strip(',').strip('.'))
    # Start with 'Meet ' and the fisrt letter of the name is uppercase
    elif text.startswith('Meet ') and re.match(r'[A-Z].*', text.split()[1]):
        dog_names.append(text.split()[1].strip(',').strip('.'))
    # Start with 'Say hello to ' and the fisrt letter of the name is uppercase
    elif text.startswith('Say hello to ') and re.match(r'[A-Z].*', text.split()[3]):
        dog_names.append(text.split()[3].strip(',').strip('.'))
    # Start with 'Here we have ' and the fisrt letter of the name is uppercase
    elif text.startswith('Here we have ') and re.match(r'[A-Z].*', text.split()[3]):
        dog_names.append(text.split()[3].strip(',').strip('.'))
    # Contain 'named' and the fisrt letter of the name is uppercase
    elif 'named' in text and re.match(r'[A-Z].*', text.split()[text.split().index('named') + 1]):
        dog_names.append(text.split()[text.split().index('named') + 1].strip(',').strip('.'))
    # No name specified or other style
    else:
        dog_names.append('NaN')

# Test
len(dog_names)

1994

In [47]:
# Save the result in a new column 'dog_name'
df_master['dog_name'] = dog_names

# Test
print("New column dog_name count \n", df_master.dog_name.value_counts())
print("Old column name count \n", df_master.name.value_counts())

New column dog_name count 
 NaN         625
Charlie      11
Cooper       10
Lucy         10
Oliver       10
Tucker        9
Penny         9
Sadie         8
Winston       8
Daisy         7
Lola          7
Bella         6
Stanley       6
Toby          6
Koda          6
Bo            6
Jax           6
Scout         5
Oscar         5
Buddy         5
Leo           5
Chester       5
Rusty         5
Milo          5
Bailey        5
Louis         5
Maggie        4
Bear          4
Clarence      4
Dexter        4
           ... 
Wafer         1
Molly         1
Harry         1
Emma          1
Philbert      1
Leela         1
Stewie        1
Wiggles       1
Lizzie        1
Cora          1
Buddah        1
Samsom        1
Chelsea       1
River         1
Nimbus        1
Noah          1
Perry         1
BeBe          1
Roscoe        1
Murphy        1
Duchess       1
Tug           1
Batdog        1
Shiloh        1
Butter        1
Divine        1
Joshwa        1
Bauer         1
Reagan        1
Bronte      

In [48]:
# We can see here that the 'NaN' result for the tweets with two names or no name
df_master[df_master.dog_name == 'NaN']

,tweet_id,timestamp,source,text,expanded_urls,name,jpg_url,favorite_count,retweet_count,dog_stage,prediction_algorithm,confidence_level,rating_numerator,rating_denominator,dogs_count,dog_name
1641,684122891630342144,2016-01-04 21:23:02 +0000,Twitter for iPhone,Heartwarming scene of two pups that want nothi...,https://twitter.com/dog_rates/status/684122891...,None,https://pbs.twimg.com/media/CX5-HslWQAIiXKB.jpg,2176.0,539.0,,NaN,0.000000,11,10,1,NaN
1638,684188786104872960,2016-01-05 01:44:52 +0000,Twitter for iPhone,"""Yo Boomer I'm taking a selfie, grab your stic...",https://twitter.com/dog_rates/status/684188786...,None,https://pbs.twimg.com/media/CX66EiJWkAAVjA-.jpg,3831.0,1343.0,,kelpie,0.537782,10,10,1,NaN
1636,684200372118904832,2016-01-05 02:30:55 +0000,Twitter for iPhone,Gang of fearless hoofed puppers here. Straight...,https://twitter.com/dog_rates/status/684200372...,None,https://pbs.twimg.com/media/CX7EkuHWkAESLZk.jpg,2394.0,1177.0,,NaN,0.000000,6,10,1,NaN
1635,684222868335505415,2016-01-05 04:00:18 +0000,Twitter for iPhone,Someone help the girl is being mugged. Several...,https://twitter.com/dog_rates/status/684222868...,None,https://pbs.twimg.com/media/CX7Y_ByWwAEJdUy.jpg,4225.0,1563.0,,soft-coated_wheaten_terrier,0.791182,11,110,11,NaN
1634,684225744407494656,2016-01-05 04:11:44 +0000,Twitter for iPhone,"Two sneaky puppers were not initially seen, mo...",https://twitter.com/dog_rates/status/684225744...,None,https://pbs.twimg.com/media/CX7br3HWsAAQ9L1.jpg,1369.0,239.0,,golden_retriever,0.203249,11,130,13,NaN
1630,684538444857667585,2016-01-06 00:54:18 +0000,Twitter for iPhone,"After watching this video, we've determined th...",https://twitter.com/dog_rates/status/684538444...,None,https://pbs.twimg.com/ext_tw_video_thumb/68453...,2915.0,1085.0,,Chihuahua,0.702583,12,10,1,NaN
1627,684594889858887680,2016-01-06 04:38:35 +0000,Twitter for iPhone,"""FOR THE LAST TIME I DON'T WANNA PLAY TWISTER ...",https://twitter.com/dog_rates/status/684594889...,None,https://pbs.twimg.com/media/CYAra7JWsAACPZH.jpg,9843.0,4016.0,,Weimaraner,0.948688,10,10,1,NaN
1622,684914660081053696,2016-01-07 01:49:14 +0000,Twitter for iPhone,"""Hello yes I'll just get one of each color tha...",https://twitter.com/dog_rates/status/684914660...,None,https://pbs.twimg.com/media/CYFOP6cWEAAWp-k.jpg,3848.0,1662.0,,chow,0.261288,12,10,1,NaN
1613,685315239903100929,2016-01-08 04:21:00 +0000,Twitter for iPhone,I would like everyone to appreciate this pup's...,https://twitter.com/dog_rates/status/685315239...,None,https://pbs.twimg.com/media/CYK6kf0WMAAzP-0.jpg,3676.0,1234.0,,chow,0.470162,11,10,1,NaN
1611,685325112850124800,2016-01-08 05:00:14 +0000,Twitter for iPhone,"""Tristan do not speak to me with that kind of ...",https://twitter.com/dog_rates/status/685325112...,None,https://pbs.twimg.com/media/CYLDikFWEAAIy1y.jpg,10471.0,4535.0,,golden_retriever,0.586937,10,10,1,NaN


In [49]:
# Let's delete the old name column now
df_master = df_master.drop(['name'], 1)

**4.8.3 TEST CLEANED ISSUE #8**

In [50]:
# Test
list(df_master)

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'jpg_url',
 'favorite_count',
 'retweet_count',
 'dog_stage',
 'prediction_algorithm',
 'confidence_level',
 'rating_numerator',
 'rating_denominator',
 'dogs_count',
 'dog_name']

### Convert the Null values to None type

<a id='issue9'></a>
**4.9.1 DEFINE ISSUE #9**

Search for **NULL** values and replace it with **NONE** value.

**4.9.2 CLEAN ISSUE #9**

In [51]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 961
Data columns (total 15 columns):
tweet_id                1994 non-null int64
timestamp               1994 non-null object
source                  1994 non-null object
text                    1994 non-null object
expanded_urls           1994 non-null object
jpg_url                 1994 non-null object
favorite_count          1994 non-null float64
retweet_count           1994 non-null float64
dog_stage               1994 non-null object
prediction_algorithm    1994 non-null object
confidence_level        1994 non-null float64
rating_numerator        1994 non-null object
rating_denominator      1994 non-null object
dogs_count              1994 non-null object
dog_name                1994 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 249.2+ KB


In [52]:
df_master.loc[df_master['dog_name'] == 'NaN', 'dog_name'] = None
df_master.loc[df_master['rating_numerator'] == 'NaN', 'rating_numerator'] = 0
df_master.loc[df_master['rating_numerator'] == 'Error', 'rating_numerator'] = 0
df_master.loc[df_master['rating_denominator'] == 'NaN', 'rating_denominator'] = 0

**4.9.3 TEST CLEANED ISSUE #9**

In [53]:
# Test
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 961
Data columns (total 15 columns):
tweet_id                1994 non-null int64
timestamp               1994 non-null object
source                  1994 non-null object
text                    1994 non-null object
expanded_urls           1994 non-null object
jpg_url                 1994 non-null object
favorite_count          1994 non-null float64
retweet_count           1994 non-null float64
dog_stage               1994 non-null object
prediction_algorithm    1994 non-null object
confidence_level        1994 non-null float64
rating_numerator        1994 non-null object
rating_denominator      1994 non-null object
dogs_count              1994 non-null object
dog_name                1369 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 249.2+ KB


### Convert each column to its appropriate type

<a id='issue10'></a>
**4.10.1 DEFINE ISSUE #10**

Replace the type of data with the appropriate type.

**4.10.2 CLEAN ISSUE #10**

In [54]:
df_master.dtypes

tweet_id                  int64
timestamp                object
source                   object
text                     object
expanded_urls            object
jpg_url                  object
favorite_count          float64
retweet_count           float64
dog_stage                object
prediction_algorithm     object
confidence_level        float64
rating_numerator         object
rating_denominator       object
dogs_count               object
dog_name                 object
dtype: object

In [55]:
df_master['tweet_id'] = df_master['tweet_id'].astype(object)
df_master['timestamp'] = pd.to_datetime(df_master.timestamp)
df_master['source'] = df_master['source'].astype('category')
df_master['dog_stage'] = df_master['dog_stage'].astype('category')
df_master['confidence_level'] = df_master['confidence_level'].astype(float)

**4.10.3 TEST CLEANED ISSUE #10**

In [56]:
# Test
df_master.dtypes

tweet_id                        object
timestamp               datetime64[ns]
source                        category
text                            object
expanded_urls                   object
jpg_url                         object
favorite_count                 float64
retweet_count                  float64
dog_stage                     category
prediction_algorithm            object
confidence_level               float64
rating_numerator                object
rating_denominator              object
dogs_count                      object
dog_name                        object
dtype: object

### Rename columns to be more expressive *and Clean if needed*

<a id='issue11'></a>
**4.11.1 DEFINE ISSUE #11**

I will replace the columns name and delete the useless columns.

**4.11.2 CLEAN ISSUE #11**

In [57]:
df_master = df_master.rename(columns = {'timestamp': 'tweet_date', 'source': 'tweet_source', 'text': 'tweet_text', 
                                        'expanded_urls': 'tweet_url', 'jpg_url': 'tweet_picture_predicted', 
                                        'favorite_count': 'tweet_favorites', 'retweet_count': 'tweet_retweets',
                                        'prediction_algorithm' : 'dog_breed'})

In [58]:
# All rating_denominator has one value 10
# We will delete this column
df_master.drop('rating_denominator', 1, inplace = True)

**4.11.3 TEST CLEANED ISSUE #11**

In [59]:
# Test
list(df_master)

['tweet_id',
 'tweet_date',
 'tweet_source',
 'tweet_text',
 'tweet_url',
 'tweet_picture_predicted',
 'tweet_favorites',
 'tweet_retweets',
 'dog_stage',
 'dog_breed',
 'confidence_level',
 'rating_numerator',
 'dogs_count',
 'dog_name']

### Additional Quality Treatment

<a id='additional'></a>
**4.12 Additional**

#### Get Dogs gender column from text column

In [60]:
# Loop on all the text and check if it has one of pronouns of male or female and append the result in a list

male = ['He', 'he', 'him', 'his', "he's", 'himself']
female = ['She', 'she', 'her', 'hers', 'herself', "she's"]

dog_gender = []

for text in df_master['tweet_text']:
    # Male
    if any(map(lambda v:v in male, text.split())):
        dog_gender.append('male')
    # Female
    elif any(map(lambda v:v in female, text.split())):
        dog_gender.append('female')
    # If group or not specified
    else:
        dog_gender.append('NaN')

# Test
len(dog_gender)

1994

In [61]:
# Save the result in a new column 'dog_name'
df_master['dog_gender'] = dog_gender

# Test
print("dog_gender count \n", df_master.dog_gender.value_counts())

dog_gender count 
 NaN       1132
male       636
female     226
Name: dog_gender, dtype: int64


#### Storing, Analyzing, and Visualizing Data

In [62]:
# Store the clean DataFrame in a CSV file
df_master.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')

In [63]:
df_master = pd.read_csv('twitter_archive_master.csv')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 15 columns):
tweet_id                   1994 non-null int64
tweet_date                 1994 non-null object
tweet_source               1994 non-null object
tweet_text                 1994 non-null object
tweet_url                  1994 non-null object
tweet_picture_predicted    1994 non-null object
tweet_favorites            1994 non-null float64
tweet_retweets             1994 non-null float64
dog_stage                  306 non-null object
dog_breed                  1686 non-null object
confidence_level           1994 non-null float64
rating_numerator           1994 non-null float64
dogs_count                 1994 non-null object
dog_name                   1369 non-null object
dog_gender                 862 non-null object
dtypes: float64(4), int64(1), object(10)
memory usage: 233.8+ KB


 The **visualizations** in the act_report.ipynd notebook in the same zip file